In [4]:
import os
import string
import annoy
import codecs

from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import Word2Vec

import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
import re

from nltk.corpus import stopwords

from string import punctuation
exclude = set(punctuation)
sw = set(get_stop_words("ru"))
morpher = MorphAnalyzer()

## Первоначальная обработка данных

In [5]:
data = pd.read_csv('./data/ProductsDataset.csv')
data

,title,descrirption,product_id,category_id,subcategory_id,properties,image_links
0,Юбка детская ORBY,"Новая, не носили ни разу. В реале красивей чем...",58e3cfe6132ca50e053f5f82,22.0,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}",http://cache3.youla.io/files/images/360_360/58...
1,Ботильоны,"Новые,привезены из Чехии ,указан размер 40,но ...",5667531b2b7f8d127d838c34,9.0,902,"{'zhenskaya_odezhda_tzvet': 'Зеленый', 'visota...",http://cache3.youla.io/files/images/360_360/5b...
2,Брюки,Размер 40-42. Брюки почти новые - не знаю как ...,59534826aaab284cba337e06,9.0,906,{'zhenskaya_odezhda_dzhinsy_bryuki_tip': 'Брюк...,http://cache3.youla.io/files/images/360_360/59...
3,Продам детские шапки,"Продам шапки,кажда 200р.Розовая и белая проданны.",57de544096ad842e26de8027,22.0,2217,"{'detskie_pol': 'Девочкам', 'detskaya_odezhda_...",http://cache3.youla.io/files/images/360_360/57...
4,Блузка,"Темно-синяя, 42 размер,состояние отличное,как ...",5ad4d2626c86cb168d212022,9.0,907,"{'zhenskaya_odezhda_tzvet': 'Синий', 'zhenskay...",http://cache3.youla.io/files/images/360_360/5a...
...,...,...,...,...,...,...,...
35543,Юбка,Юбка Белая по.Турция фирма adL,5b5f181c62e1c6616a7f6472,9.0,904,"{'zhenskaya_odezhda_platya_yubki_tip': 'Юбки',...",http://cache3.youla.io/files/images/360_360/5b...
35544,Новый твидовый пиджак,Новый с бирками пиджак размер S в стиле Coco C...,5bd6c8b29e94ba033d31f8d0,9.0,908,"{'brand_zhenskii': 'Chanel', 'zhenskaya_odezhd...",http://cache3.youla.io/files/images/360_360/5b...
35545,Женская зимняя куртка,Женская зимняя спортивная куртка фирмы Rossiqn...,5bd6c8bc074b3e1c056f69b2,9.0,903,"{'zhenskaya_odezhda_razmer': '48-50 (XL)', 'zh...",http://cache3.youla.io/files/images/360_360/5b...
35546,Новая золотая ветровка,Женская ветровка размер 44-46. Цвет приглушённ...,5bd6c8fb2138bbc55745362c,9.0,903,"{'zhenskaya_odezhda_razmer': '44-46 (М)', 'zhe...",http://cache3.youla.io/files/images/360_360/5b...


Проверим и удалим пропуски данных. Если нет признаков то и нельзя детерминировать объект

In [6]:
data.isnull().sum(axis = 0)
data.dropna( inplace=True)

In [7]:
data.isnull().sum(axis = 0)

title             0
descrirption      0
product_id        0
category_id       0
subcategory_id    0
properties        0
image_links       0
dtype: int64

Преобразуем данные к типам

In [8]:
data['category_id'] = data['category_id'].astype(int)
data['subcategory_id'] = data['subcategory_id'].astype(int)

 Удаляем знаки препинания, стоп слова и делаем лемматизацию

In [9]:
import re
def preprocess_txt(line):
    line = str(line).lower()
    line = re.sub(",", " ", line)
    line = re.sub("\.", " ", line)
    line = re.sub("-", " ", line)
     
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [i for i in spls if i not in sw and i != ""]
    # удалим число как мало инфомративное свойство
    spls = [i for i in spls if not str(i).isnumeric()]

    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = " ".join(i for i in spls )
    return spls

data['feature'] = data['descrirption'].apply(preprocess_txt)
data['target'] = data['title'].apply(preprocess_txt)

In [10]:
data

,title,descrirption,product_id,category_id,subcategory_id,properties,image_links,feature,target
0,Юбка детская ORBY,"Новая, не носили ни разу. В реале красивей чем...",58e3cfe6132ca50e053f5f82,22,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}",http://cache3.youla.io/files/images/360_360/58...,новый носить раз реал красивый фото,юбка детский orby
1,Ботильоны,"Новые,привезены из Чехии ,указан размер 40,но ...",5667531b2b7f8d127d838c34,9,902,"{'zhenskaya_odezhda_tzvet': 'Зеленый', 'visota...",http://cache3.youla.io/files/images/360_360/5b...,новый привезти чехия указать размер маломерить...,ботильон
2,Брюки,Размер 40-42. Брюки почти новые - не знаю как ...,59534826aaab284cba337e06,9,906,{'zhenskaya_odezhda_dzhinsy_bryuki_tip': 'Брюк...,http://cache3.youla.io/files/images/360_360/59...,размер брюки новый знать мерило покупка носить...,брюки
3,Продам детские шапки,"Продам шапки,кажда 200р.Розовая и белая проданны.",57de544096ad842e26de8027,22,2217,"{'detskie_pol': 'Девочкам', 'detskaya_odezhda_...",http://cache3.youla.io/files/images/360_360/57...,продать шапка кажда 200р розовый белый проданна,продать детский шапка
4,Блузка,"Темно-синяя, 42 размер,состояние отличное,как ...",5ad4d2626c86cb168d212022,9,907,"{'zhenskaya_odezhda_tzvet': 'Синий', 'zhenskay...",http://cache3.youla.io/files/images/360_360/5a...,темно синий размер состояние отличный новый см...,блузка
...,...,...,...,...,...,...,...,...,...
35542,Новое болеро s'Oliver,"Привезли из Европы, не подошло по размеру\n\nР...",5ac078a7938000715c0e2c38,9,908,{'zhenskaya_odezhda_pidzhaki_kostyumy_tip': 'П...,http://cache3.youla.io/files/images/360_360/5a...,привезти европа подойти размер разгружать гард...,новый болеро soliver
35543,Юбка,Юбка Белая по.Турция фирма adL,5b5f181c62e1c6616a7f6472,9,904,"{'zhenskaya_odezhda_platya_yubki_tip': 'Юбки',...",http://cache3.youla.io/files/images/360_360/5b...,юбка белый турция фирма adl,юбка
35544,Новый твидовый пиджак,Новый с бирками пиджак размер S в стиле Coco C...,5bd6c8b29e94ba033d31f8d0,9,908,"{'brand_zhenskii': 'Chanel', 'zhenskaya_odezhd...",http://cache3.youla.io/files/images/360_360/5b...,новый бирка пиджак размер s стиль coco chanel,новый твидовый пиджак
35545,Женская зимняя куртка,Женская зимняя спортивная куртка фирмы Rossiqn...,5bd6c8bc074b3e1c056f69b2,9,903,"{'zhenskaya_odezhda_razmer': '48-50 (XL)', 'zh...",http://cache3.youla.io/files/images/360_360/5b...,женский зимний спортивный куртка фирма rossiqn...,женский зимний куртка


Задача - предсказать описание товара или произвольный текст. Для это выделим данные из болталки

In [8]:
question = None
written = False

#Мы идем по всем записям, берем первую строку как вопрос
# и после знака --- находим ответ
with codecs.open("./data/prepared_answers.txt","w", "utf-8") as fout:
    with codecs.open("./data/Otvety.txt", "r", "utf-8") as fin:
        for line in tqdm(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

0it [00:00, ?it/s]

In [16]:
sentences = []
c = 0

with codecs.open("./data/Otvety.txt", "r", "utf-8") as fin:
    for line in tqdm(fin):
        spls = preprocess_txt(line)
        if len(spls):
            sentences.append(spls)
            c += 1
        if c > 100000:
            break

0it [00:00, ?it/s]

In [17]:
data.shape

(33531, 9)

Обьединим данные описание товара и произвольный текст в единый датасет. Для балансировки датасетов возьмем из болталки колисчество данных равных в датасете с описанием товаров

In [27]:
import random
botSelect= random.sample(sentences, data.shape[0])

botData =  pd.DataFrame()
botData["text"] = botSelect
botData["target"] = 0

clissifyData = data[["feature"]].copy()
clissifyData.rename({'feature': 'text'}, axis=1, inplace=True)
clissifyData["target"] = 1

clissify = pd.concat([botData, clissifyData], ignore_index=True, axis=0).copy()

In [28]:
clissify

,text,target
0,идея создание чётный нечётный колода hearthcto...,0
1,знать советский попасться пенять,0
2,эпсилон любой положительный число сказать любо...,0
3,мяукать,0
4,окно открывать долгоbrкирпич заложеноbrжить за...,0
...,...,...
67057,привезти европа подойти размер разгружать гард...,1
67058,юбка белый турция фирма adl,1
67059,новый бирка пиджак размер s стиль coco chanel,1
67060,женский зимний спортивный куртка фирма rossiqn...,1


In [29]:
from sklearn import model_selection, preprocessing
X_train, X_test, y_train, y_test = model_selection.train_test_split(clissify['text'], clissify['target'])

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizerGram = TfidfVectorizer( )
vectorizerGram.fit(X_train.values)

X_train = vectorizerGram.transform(X_train)
y_train = y_train

X_test = vectorizerGram.transform(X_test)
y_test = y_test


In [31]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs' )
model.fit(X_train, y_train)

LogisticRegression()

In [32]:
predictions = model.predict(X_test)

In [33]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, predictions)


0.9712513420016701

In [34]:
import pickle
def writeModel(model, filename):
    pickle.dump(model, open(filename, 'wb'))


def readModel(model, filename):
    loaded_model = pickle.load(open(filename, 'rb'))
    return loaded_model

In [35]:
filename = './model/classify.sav'
writeModel(model,filename)

Наша модель с точностью 97 процентов правильно классифицирует текст

### 2 Реализован поиск похожих товаров в контентной части бота

Для этого обучим модель Word2Vec, соединив колонки   feature и  target


In [70]:
goodSimilar = data[['product_id','title', 'descrirption']].copy()
goodSimilar['text'] =  goodSimilar['title']

In [71]:
goodSimilar

,product_id,title,descrirption,text
0,58e3cfe6132ca50e053f5f82,Юбка детская ORBY,"Новая, не носили ни разу. В реале красивей чем...",Юбка детская ORBY
1,5667531b2b7f8d127d838c34,Ботильоны,"Новые,привезены из Чехии ,указан размер 40,но ...",Ботильоны
2,59534826aaab284cba337e06,Брюки,Размер 40-42. Брюки почти новые - не знаю как ...,Брюки
3,57de544096ad842e26de8027,Продам детские шапки,"Продам шапки,кажда 200р.Розовая и белая проданны.",Продам детские шапки
4,5ad4d2626c86cb168d212022,Блузка,"Темно-синяя, 42 размер,состояние отличное,как ...",Блузка
...,...,...,...,...
35542,5ac078a7938000715c0e2c38,Новое болеро s'Oliver,"Привезли из Европы, не подошло по размеру\n\nР...",Новое болеро s'Oliver
35543,5b5f181c62e1c6616a7f6472,Юбка,Юбка Белая по.Турция фирма adL,Юбка
35544,5bd6c8b29e94ba033d31f8d0,Новый твидовый пиджак,Новый с бирками пиджак размер S в стиле Coco C...,Новый твидовый пиджак
35545,5bd6c8bc074b3e1c056f69b2,Женская зимняя куртка,Женская зимняя спортивная куртка фирмы Rossiqn...,Женская зимняя куртка


In [72]:
def preprocessTxtWec2(line):
    line = str(line).lower()
    line = re.sub(",", " ", line)
    line = re.sub("\.", " ", line)
    line = re.sub("-", " ", line)

    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [i for i in spls if i not in sw and i != ""]
    # удалим число как мало инфомративное свойство
    spls = [i for i in spls if not str(i).isnumeric()]

    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    return spls
goodSimilar['text'] = goodSimilar['text'].apply(preprocessTxtWec2)

In [73]:
silimarTrain = list(goodSimilar['text'].values)
silimarTrain = [i for i in silimarTrain if len(i) > 2]

In [74]:
model = Word2Vec(sentences=silimarTrain, vector_size=100, min_count=1, window=5)
model.save("./model/similar_model")

In [186]:
# goodSimilar.to_csv('./data/ProductsDatasetPrepare.csv', index=False)

In [75]:
model = Word2Vec.load('./model/similar_model')
indexBase = annoy.AnnoyIndex(100 ,'angular')

index_map = {}
counter = 0

for index, row in goodSimilar.iterrows():
    n_w2v = 0
    prodictId = row['product_id']
    index_map[counter] = prodictId

    productName = preprocess_txt(row['title'])

    vector = np.zeros(100)
    for word in productName:
        if word in model.wv:
            vector += model.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    indexBase.add_item(counter, vector)
    counter += 1
    

indexBase.build(10)
indexBase.save('./model/similar.ann')

True

In [76]:
model = Word2Vec.load('./model/similar_model')
threshold = 0.02
def get_answer (question):
    preprocessed_question = preprocess_txt(question)
    n_w2v = 0
    vector = np.zeros(100)
    for word in preprocessed_question:
        if word in model.wv:
            vector += model.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    answer_index = indexBase.get_nns_by_vector(vector, 5, include_distances=True)
    
    asnwer = {}
    
    index = answer_index[0]
    distance = answer_index[1]
    for index, item in enumerate(index):
        if distance[index] <= threshold:
            productId = index_map[item]
            asnwer[productId] = data[data["product_id"] == productId]['title'].values[0]

    return asnwer

In [77]:
get_answer(" Юбка детская ORBY")

{'58e3cfe6132ca50e053f5f82': 'Юбка детская ORBY'}

In [78]:
get_answer(" Где ключи от танка")

{'5ab1054baaab283bcf7d0ce9': 'Костюмчик на весну, лето'}

### 2.1 Реализуем поиск  похожих товаров по описанию

In [79]:
goodSimilar = data[['title', 'descrirption']].copy()
goodSimilar['text'] =  goodSimilar['descrirption']
goodSimilar

,title,descrirption,text
0,Юбка детская ORBY,"Новая, не носили ни разу. В реале красивей чем...","Новая, не носили ни разу. В реале красивей чем..."
1,Ботильоны,"Новые,привезены из Чехии ,указан размер 40,но ...","Новые,привезены из Чехии ,указан размер 40,но ..."
2,Брюки,Размер 40-42. Брюки почти новые - не знаю как ...,Размер 40-42. Брюки почти новые - не знаю как ...
3,Продам детские шапки,"Продам шапки,кажда 200р.Розовая и белая проданны.","Продам шапки,кажда 200р.Розовая и белая проданны."
4,Блузка,"Темно-синяя, 42 размер,состояние отличное,как ...","Темно-синяя, 42 размер,состояние отличное,как ..."
...,...,...,...
35542,Новое болеро s'Oliver,"Привезли из Европы, не подошло по размеру\n\nР...","Привезли из Европы, не подошло по размеру\n\nР..."
35543,Юбка,Юбка Белая по.Турция фирма adL,Юбка Белая по.Турция фирма adL
35544,Новый твидовый пиджак,Новый с бирками пиджак размер S в стиле Coco C...,Новый с бирками пиджак размер S в стиле Coco C...
35545,Женская зимняя куртка,Женская зимняя спортивная куртка фирмы Rossiqn...,Женская зимняя спортивная куртка фирмы Rossiqn...


In [80]:
goodSimilar['text'] = goodSimilar['text'].apply(preprocessTxtWec2)

In [81]:
goodSimilar

,title,descrirption,text
0,Юбка детская ORBY,"Новая, не носили ни разу. В реале красивей чем...","[новый, носить, раз, реал, красивый, фото]"
1,Ботильоны,"Новые,привезены из Чехии ,указан размер 40,но ...","[новый, привезти, чехия, указать, размер, мало..."
2,Брюки,Размер 40-42. Брюки почти новые - не знаю как ...,"[размер, брюки, новый, знать, мерило, покупка,..."
3,Продам детские шапки,"Продам шапки,кажда 200р.Розовая и белая проданны.","[продать, шапка, кажда, 200р, розовый, белый, ..."
4,Блузка,"Темно-синяя, 42 размер,состояние отличное,как ...","[темно, синий, размер, состояние, отличный, но..."
...,...,...,...
35542,Новое болеро s'Oliver,"Привезли из Европы, не подошло по размеру\n\nР...","[привезти, европа, подойти, размер, разгружать..."
35543,Юбка,Юбка Белая по.Турция фирма adL,"[юбка, белый, турция, фирма, adl]"
35544,Новый твидовый пиджак,Новый с бирками пиджак размер S в стиле Coco C...,"[новый, бирка, пиджак, размер, s, стиль, coco,..."
35545,Женская зимняя куртка,Женская зимняя спортивная куртка фирмы Rossiqn...,"[женский, зимний, спортивный, куртка, фирма, r..."


In [84]:
silimarProduct = list(goodSimilar['text'].values)
silimarProduct

[['новый', 'носить', 'раз', 'реал', 'красивый', 'фото'],
 ['новый',
  'привезти',
  'чехия',
  'указать',
  'размер',
  'маломерить',
  'каблук',
  'смотреть',
  'небольшой',
  'дефект',
  'каблук'],
 ['размер',
  'брюки',
  'новый',
  'знать',
  'мерило',
  'покупка',
  'носить',
  'смочь',
  'узкий',
  'пояс',
  'покупать',
  'отдать',
  'подойти',
  'высокий',
  'рост',
  'подшить',
  'носить',
  'офис',
  'выход',
  '1😊',
  'качество',
  'хороший',
  'турция'],
 ['продать', 'шапка', 'кажда', '200р', 'розовый', 'белый', 'проданна'],
 ['темно',
  'синий',
  'размер',
  'состояние',
  'отличный',
  'новый',
  'смотреть',
  'профиль',
  'интересный',
  'народный',
  'улица'],
 ['отличный', 'состояние', 'фирма', 'koton', 'р', 'смотреть', 'профиль'],
 ['женский', 'ботинок', 'topshop', 'заклёпка', 'кожзам', 'размер'],
 ['туфля', 'кожа', 'размер', 'заходить', 'профиль'],
 ['одеть',
  'концерт',
  'размер',
  'сидеть',
  'фигура',
  'красиво',
  'нежно',
  'сзади',
  'молния',
  'рост',
  '

In [85]:
model = Word2Vec(sentences=silimarProduct, vector_size=100, min_count=1, window=5)
model.save("./model/similar_product")

In [86]:
goodSimilar

,title,descrirption,text
0,Юбка детская ORBY,"Новая, не носили ни разу. В реале красивей чем...","[новый, носить, раз, реал, красивый, фото]"
1,Ботильоны,"Новые,привезены из Чехии ,указан размер 40,но ...","[новый, привезти, чехия, указать, размер, мало..."
2,Брюки,Размер 40-42. Брюки почти новые - не знаю как ...,"[размер, брюки, новый, знать, мерило, покупка,..."
3,Продам детские шапки,"Продам шапки,кажда 200р.Розовая и белая проданны.","[продать, шапка, кажда, 200р, розовый, белый, ..."
4,Блузка,"Темно-синяя, 42 размер,состояние отличное,как ...","[темно, синий, размер, состояние, отличный, но..."
...,...,...,...
35542,Новое болеро s'Oliver,"Привезли из Европы, не подошло по размеру\n\nР...","[привезти, европа, подойти, размер, разгружать..."
35543,Юбка,Юбка Белая по.Турция фирма adL,"[юбка, белый, турция, фирма, adl]"
35544,Новый твидовый пиджак,Новый с бирками пиджак размер S в стиле Coco C...,"[новый, бирка, пиджак, размер, s, стиль, coco,..."
35545,Женская зимняя куртка,Женская зимняя спортивная куртка фирмы Rossiqn...,"[женский, зимний, спортивный, куртка, фирма, r..."


In [87]:
modelProduct = Word2Vec.load('./model/similar_product')
indexProduct = annoy.AnnoyIndex(100 ,'angular')

index_product = {}
counter = 0

for index, row in goodSimilar.iterrows():
    n_w2v = 0
    title = row['title']
    index_product[counter] = title

    descrirption = preprocess_txt(row['descrirption'])

    vector = np.zeros(100)
    for word in descrirption:
        if word in modelProduct.wv:
            vector += modelProduct.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    indexProduct.add_item(counter, vector)
    counter += 1
    

indexProduct.build(10)
indexProduct.save('./model/similarProduct.ann')

True

In [97]:
modelProduct = Word2Vec.load('./model/similar_product')

threshold = 0.05

def get_answer_product (question):
    preprocessed_question = preprocess_txt(question)
    n_w2v = 0
    vector = np.zeros(100)
    for word in preprocessed_question:
        if word in modelProduct.wv:
            vector += modelProduct.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    answer_index = indexProduct.get_nns_by_vector(vector, 5, include_distances=True)
    
    asnwer = {}    
    index = answer_index[0]
    distance = answer_index[1]
    for index, item in enumerate(index):
        if distance[index] <= threshold:
            productName = index_product[item]
            asnwer[item] = productName

    return asnwer

In [101]:
get_answer_product('Юбка Белая по.Турция фирма adL')

{33527: 'Юбка'}

Вывод: если поиск по названию еще работает, то поиск товара по его описанию не очень. Видимо надо как то еще описание разделять!

# 3 Реализована болталка

In [103]:
sentences = []
c = 0
with codecs.open("./data/Otvety.txt", "r", "utf-8") as fin:
    for line in tqdm(fin):
        spls = preprocess_txt(line)
        if len(spls):
            sentences.append(spls)
            c += 1
        if c > 500000:
            break
sentences = [i for i in sentences if len(i) > 2]

0it [00:00, ?it/s]

In [106]:
model = Word2Vec(sentences=sentences, vector_size=100, min_count=1, window=5)
model.save('./model/bot_model')

In [107]:
indexBot = annoy.AnnoyIndex(100 ,'angular')
modelBot = Word2Vec.load('./model/bot_model')

index_bot_map = {}
counter = 0

with codecs.open("./data/prepared_answers.txt", "r", "utf-8") as f:
    for line in tqdm(f):
        n_w2v = 0
        spls = line.split("\t")
        index_bot_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        vector = np.zeros(100)
        for word in question:
            if word in modelBot.wv:
                vector += modelBot.wv[word]
                n_w2v += 1
        if n_w2v > 0:
            vector = vector / n_w2v
        indexBot.add_item(counter, vector)
            
        counter += 1

indexBot.build(10)
indexBot.save('./model/speaker.ann')

0it [00:00, ?it/s]

True

In [108]:
def find_answer(question):
    preprocessed_question = preprocess_txt(question)
    n_w2v = 0
    vector = np.zeros(100)
    for word in preprocessed_question:
        if word in modelBot.wv:
            vector += modelBot.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    answer_index = indexBot.get_nns_by_vector(vector, 1)
    return index_bot_map[answer_index[0]]

In [112]:
find_answer('есть ли дурак')

'Смотря для кого.... \n'

Вывод
 - реализивана классификация тексте
 - реализовано на базе алгорпитма Word2Vec  поиск товаров по названию и поиск товаров по описанию
 - реализована болталка (по факту изучена с прошлого урока и с помощью этой реализации были получены результаты в предудущем пункет)
 
 По факту в юните изучены принципы векторизации текста,  методы классификации текста, метод Word2Vec для векторизации текста